In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load data

In [ ]:
sells_train = pd.read_csv("sales_train_validation.csv")
sells_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [ ]:
sells_train.shape

(30490, 1919)

In [ ]:
calendar = pd.read_csv("calendar.csv")
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
calendar.shape

(1969, 14)

In [ ]:
sell_prices = pd.read_csv("sell_prices.csv")
print(sell_prices.shape)
sell_prices.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
sell_prices['wm_yr_wk'].unique()

282

In [ ]:
sell_prices.set_index(['store_id', 'item_id', 'wm_yr_wk'], inplace=True)
sell_prices.head()

sell_price
store_id item_id       wm_yr_wk            
CA_1     HOBBIES_1_001 11325           9.58
                       11326           9.58
                       11327           8.26
                       11328           8.26
                       11329           8.26

In [ ]:
sell_prices.index

MultiIndex([('CA_1', 'HOBBIES_1_001', 11325),
            ('CA_1', 'HOBBIES_1_001', 11326),
            ('CA_1', 'HOBBIES_1_001', 11327),
            ('CA_1', 'HOBBIES_1_001', 11328),
            ('CA_1', 'HOBBIES_1_001', 11329),
            ('CA_1', 'HOBBIES_1_001', 11330),
            ('CA_1', 'HOBBIES_1_001', 11331),
            ('CA_1', 'HOBBIES_1_001', 11332),
            ('CA_1', 'HOBBIES_1_001', 11333),
            ('CA_1', 'HOBBIES_1_001', 11334),
            ...
            ('WI_3',   'FOODS_3_827', 11612),
            ('WI_3',   'FOODS_3_827', 11613),
            ('WI_3',   'FOODS_3_827', 11614),
            ('WI_3',   'FOODS_3_827', 11615),
            ('WI_3',   'FOODS_3_827', 11616),
            ('WI_3',   'FOODS_3_827', 11617),
            ('WI_3',   'FOODS_3_827', 11618),
            ('WI_3',   'FOODS_3_827', 11619),
            ('WI_3',   'FOODS_3_827', 11620),
            ('WI_3',   'FOODS_3_827', 11621)],
           names=['store_id', 'item_id', 'wm_yr_wk'], length=68

### Build training dataframe

We expanded each product to have 1913 rows. This way, we could have historical sales data into daily sales columns, and integrate it with the calendar data.

In [ ]:
id_order = sells_train['id'].tolist()
id_order[:10]

['HOBBIES_1_001_CA_1_validation',
 'HOBBIES_1_002_CA_1_validation',
 'HOBBIES_1_003_CA_1_validation',
 'HOBBIES_1_004_CA_1_validation',
 'HOBBIES_1_005_CA_1_validation',
 'HOBBIES_1_006_CA_1_validation',
 'HOBBIES_1_007_CA_1_validation',
 'HOBBIES_1_008_CA_1_validation',
 'HOBBIES_1_009_CA_1_validation',
 'HOBBIES_1_010_CA_1_validation']

In [ ]:
day_num = 1913
basic_data = sells_train.iloc[:, :6].copy()
duplicate = [basic_data]*day_num
basic_data = pd.concat(duplicate)
basic_data['id'] = basic_data['id'].astype('category')
basic_data['id'] = basic_data['id'].cat.set_categories(id_order)
basic_data = basic_data.sort_values(by = ['id']).reset_index(drop = True)
basic_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA


In [ ]:
day_name = sells_train.columns[6:].tolist()
day_name[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [ ]:
sample_num = len(sells_train)
basic_data['d'] = day_name*sample_num
basic_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5


In [ ]:
sell_amt = np.reshape(sells_train.iloc[:, 6:].values, (day_num*sample_num, 1))
basic_data['num_sold'] = sell_amt
basic_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0


In [ ]:
basic_data.shape

(58327370, 8)

In [ ]:
cal_df = pd.concat([calendar.iloc[:1913]]*sample_num).reset_index(drop=True)
cal_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58327365,2016-04-20,11612,Wednesday,5,4,2016,d_1909,NaN,NaN,NaN,NaN,0,0,0
58327366,2016-04-21,11612,Thursday,6,4,2016,d_1910,NaN,NaN,NaN,NaN,0,0,0
58327367,2016-04-22,11612,Friday,7,4,2016,d_1911,NaN,NaN,NaN,NaN,0,0,0
58327368,2016-04-23,11613,Saturday,1,4,2016,d_1912,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
cal_df.drop(['wday','d'], axis=1, inplace = True)

In [ ]:
cal_df.shape

(58327370, 12)

In [ ]:
train_data = pd.concat([basic_data, cal_df], axis=1)
train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,2,2011,NaN,NaN,NaN,NaN,1,0,1


In [ ]:
train_data.shape

(58327370, 20)

In [ ]:
import gc
gc.collect()

100

### Snap and state

Convert the "snap" column into a single dummy variable column based on the "state".

In [ ]:
state2snap = lambda x: x['snap_CA'] if x['state_id'] == 'CA' else(x['snap_TX'] if x['state_id']=='TX' else x['snap_WI'])

In [ ]:
snap = train_data[['snap_CA', 'snap_TX', 'snap_WI', 'state_id']].apply(state2snap, axis=1)
snap

0           0
1           0
2           0
3           1
4           1
           ..
58327365    0
58327366    0
58327367    0
58327368    0
58327369    0
Length: 58327370, dtype: int64

In [ ]:
train_data.drop(['snap_CA', 'snap_TX', 'snap_WI'], inplace = True, axis=1)

In [ ]:
train_data['snap'] = snap

In [ ]:
train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,1,2011,NaN,NaN,NaN,NaN,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,1,2011,NaN,NaN,NaN,NaN,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,2,2011,NaN,NaN,NaN,NaN,1
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,2,2011,NaN,NaN,NaN,NaN,1


In [ ]:
#train_data.to_csv('train.csv',index=False)

### Merge

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
#.progress_apply()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  """Entry point for launching an IPython kernel.
/Applications/anaconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
train_data = pd.read_csv("train.csv")

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,1,2011,NaN,NaN,NaN,NaN,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,1,2011,NaN,NaN,NaN,NaN,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,2,2011,NaN,NaN,NaN,NaN,1
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,2,2011,NaN,NaN,NaN,NaN,1


In [ ]:
train_data.shape

(58327370, 18)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 18 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   id            object
 1   item_id       object
 2   dept_id       object
 3   cat_id        object
 4   store_id      object
 5   state_id      object
 6   d             object
 7   num_sold      int64 
 8   date          object
 9   wm_yr_wk      int64 
 10  weekday       object
 11  month         int64 
 12  year          int64 
 13  event_name_1  object
 14  event_type_1  object
 15  event_name_2  object
 16  event_type_2  object
 17  snap          int64 
dtypes: int64(5), object(13)
memory usage: 7.8+ GB


In [ ]:
sell_prices = pd.read_csv("sell_prices.csv")
print(sell_prices.shape)
sell_prices.head()

(6841121, 4)


,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [ ]:
sell_prices.head(10)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
5,CA_1,HOBBIES_1_001,11330,8.26
6,CA_1,HOBBIES_1_001,11331,8.26
7,CA_1,HOBBIES_1_001,11332,8.26
8,CA_1,HOBBIES_1_001,11333,8.26
9,CA_1,HOBBIES_1_001,11334,8.26


In [ ]:
train_data_withprice=train_data.merge(sell_prices,"left")

In [ ]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [ ]:
train_data_withprice.shape

(58327370, 19)

In [ ]:
train_data_withprice.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,NaN
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,1,2011,NaN,NaN,NaN,NaN,0,NaN
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,1,2011,NaN,NaN,NaN,NaN,0,NaN
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,2,2011,NaN,NaN,NaN,NaN,1,NaN
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,2,2011,NaN,NaN,NaN,NaN,1,NaN


In [ ]:
#train_data_withprice.to_csv('train_with_price_notfill.csv',index=False)


## Fill na values for "price"

In [ ]:
full_price=train_data_withprice['sell_price'].fillna(method='bfill')

In [ ]:
train_data_withprice['sell_price']=full_price

In [ ]:
train_data_withprice.head(10)

,id,item_id,dept_id,cat_id,store_id,state_id,d,num_sold,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,2011,NaN,NaN,NaN,NaN,0,9.58
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0,2011-01-30,11101,Sunday,1,2011,NaN,NaN,NaN,NaN,0,9.58
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0,2011-01-31,11101,Monday,1,2011,NaN,NaN,NaN,NaN,0,9.58
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0,2011-02-01,11101,Tuesday,2,2011,NaN,NaN,NaN,NaN,1,9.58
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0,2011-02-02,11101,Wednesday,2,2011,NaN,NaN,NaN,NaN,1,9.58
5,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_6,0,2011-02-03,11101,Thursday,2,2011,NaN,NaN,NaN,NaN,1,9.58
6,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_7,0,2011-02-04,11101,Friday,2,2011,NaN,NaN,NaN,NaN,1,9.58
7,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_8,0,2011-02-05,11102,Saturday,2,2011,NaN,NaN,NaN,NaN,1,9.58
8,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_9,0,2011-02-06,11102,Sunday,2,2011,SuperBowl,Sporting,NaN,NaN,1,9.58
9,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_10,0,2011-02-07,11102,Monday,2,2011,NaN,NaN,NaN,NaN,1,9.58


In [ ]:
#train_data_withprice.to_csv('train_with_price.csv',index=False)

### The main DataFrame is complete and can now be used in the models.


